In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import optuna

def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [2]:
train = pd.read_parquet('../src/data/raw/train.parquet')
train['monthyear'] = (pd.to_datetime(train['S_2']).dt.year.astype(str)+
                      pd.to_datetime(train['S_2']).dt.month.map("{:02}".format).astype(str))

# x = train.groupby("customer_ID")['monthyear'].agg(['min','max','nunique'])
x = train.groupby("customer_ID")['monthyear'].agg(['nunique'])

In [15]:
prefixes = ['oof_v3_dpv25fold_seed42', 'oof_v1_dpv25fold_seed98', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

for i in range(1, 14):
    lthan = amex_metric(valids[2][valids[2].customer_ID.isin(x[x['nunique']<=i].index.tolist())]['target'], 
            valids[2][valids[2].customer_ID.isin(x[x['nunique']<=i].index.tolist())]['prediction'])
    if i <13:
        gthan = amex_metric(valids[2][valids[2].customer_ID.isin(x[x['nunique']>i].index.tolist())]['target'], 
                valids[2][valids[2].customer_ID.isin(x[x['nunique']>i].index.tolist())]['prediction'])
    else: gthan = -1
    print(f'Nunique {i}: less-than score {lthan}, greater-than score{gthan}')

Nunique 1: less-than score 0.6225173217635962, greater-than score0.7996824113164427
Nunique 2: less-than score 0.6226079728530806, greater-than score0.8021679070147858
Nunique 3: less-than score 0.6346767585761496, greater-than score0.8043591929699379
Nunique 4: less-than score 0.6300117792947451, greater-than score0.8070784957568415
Nunique 5: less-than score 0.6365798250458007, greater-than score0.8088062968739889
Nunique 6: less-than score 0.6426331662786908, greater-than score0.8108833586745765
Nunique 7: less-than score 0.6485652488655076, greater-than score0.8126923011997452
Nunique 8: less-than score 0.6518806963951781, greater-than score0.8148708731962702
Nunique 9: less-than score 0.6570058871126497, greater-than score0.8166910804615415
Nunique 10: less-than score 0.659927197311278, greater-than score0.8189917307647724
Nunique 11: less-than score 0.6631892002647499, greater-than score0.8205693113790544
Nunique 12: less-than score 0.6719086819329969, greater-than score0.8231005

In [3]:
prefixes = ['oof_blend_99_98_97_cv985', 'oof_blend_99_98_97_cv985', 'oof_blend_99_98_98_97_cv798828']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

for i in range(1, 14):
    lthan = amex_metric(valids[2][valids[2].customer_ID.isin(x[x['nunique']<=i].index.tolist())]['target'], 
            valids[2][valids[2].customer_ID.isin(x[x['nunique']<=i].index.tolist())]['prediction'])
    if i <13:
        gthan = amex_metric(valids[2][valids[2].customer_ID.isin(x[x['nunique']>i].index.tolist())]['target'], 
                valids[2][valids[2].customer_ID.isin(x[x['nunique']>i].index.tolist())]['prediction'])
    else: gthan = -1
    print(f'Nunique {i}: less-than score {lthan}, greater-than score{gthan}')

KeyError: 'target'

In [17]:
valids[2][valids[2].customer_ID.isin(x[x['nunique']<=1].index.tolist())]

,customer_ID,target,prediction
63,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d...,0,0.534292
97,000eee12732c9c7b790870c556bc33339d6d17725cbeb4...,0,0.011431
145,0015d621bacbfd59cc1479aea405645ecb480f6ee779e0...,1,0.869443
149,00163cf7b8cc9cb2781ac05331d83c3ae7720f4860ee8a...,0,0.069936
497,004bf15cf65c01b2173e53c5284933d41650e69d977ebd...,1,0.839805
...,...,...,...
458639,ffd968fe02b1dfa9e26e3369e36e2c5b67fa9d42f9f926...,1,0.936909
458670,ffde43969200715fd23e7ce84e39c93931143596250ac8...,0,0.384265
458822,fff3fcf0e0c15a3069b7fba318b75c74c2729927a69cff...,1,0.994100
458856,fff845a9c5e9206a7f783640085531e10d6a9e54c2f1cc...,0,0.845327


In [27]:
x[x['nunique']==2]

,min,max,nunique
customer_ID,,,
0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab,201802,201803,2
00237aac0b2f8f6765ab8ac6ea3212afa40fab21dd3470af6f78ac4ff7b98a5b,201802,201803,2
0024753e67ff07c9b87a98b45ccb4e8a6adf509bc38d9c50a5bd86aa45339db0,201802,201803,2
0031fb6d21624e95989be3c77e7cd2ae1cb50b12739541803a78909e272086ee,201802,201803,2
0040899a63cbdfa303496756b4d3764e8e95c3d7f73723dc3672e2fcdda546e3,201802,201803,2
...,...,...,...
ffedc9ef70554110c7f5fb858b3c004373d09f19f8c37ca6517219388b9454df,201802,201803,2
fff1eb4b36987ef45a47337db49747307d633e76aca2be6d4a28e461ed159453,201802,201803,2
fffd4c8532e6746a53dfdd52de2219c847ddb0123c27ccb64cefd5291be2ed35,201802,201803,2


In [28]:
x['min'].unique()

array(['201703', '201801', '201712', '201707', '201708', '201802',
       '201706', '201803', '201709', '201704', '201710', '201705',
       '201711'], dtype=object)

In [3]:
prefixes = ['oof_v3_dpv25fold_seed42', 'oof_v1_dpv25fold_seed98', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]
prefixes = ['lgb_v2_dpv25fold_seed42', 'lgb_v1_dpv25fold_seed98', 'lgb_v2_dpv25fold_seed42']
submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

In [4]:
def objective(trial: optuna.Trial):
    
    p_lgbm = trial.suggest_uniform('lgbm', 0.0, 1.0)
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    p_xgb = trial.suggest_uniform('xgb', 0.0, 1.0)
    y = valids[0]['target'].values
    a_lgbm = valids[0].drop(['target','customer_ID'], axis=1).values
    a_cat = valids[1].drop(['target','customer_ID'], axis=1).values
    a_xgb = valids[2].drop(['target','customer_ID'], axis=1).values
    a = (p_lgbm * a_lgbm + p_cat * a_cat + p_xgb * a_xgb)
    b = a.reshape(len(a_lgbm))/(p_lgbm+p_cat+p_xgb)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=300)

[I 2022-08-12 04:00:09,321] A new study created in memory with name: no-name-b7a5cf7a-d173-4f2c-98c2-075488bc0230
[I 2022-08-12 04:00:09,598] Trial 0 finished with value: 0.798491824785957 and parameters: {'lgbm': 0.4495759331379432, 'cat': 0.9838503923925408, 'xgb': 0.7824844622901417}. Best is trial 0 with value: 0.798491824785957.
[I 2022-08-12 04:00:09,807] Trial 1 finished with value: 0.7983719573587996 and parameters: {'lgbm': 0.48314022443794036, 'cat': 0.5504138728555543, 'xgb': 0.1329073976500159}. Best is trial 0 with value: 0.798491824785957.
[I 2022-08-12 04:00:10,007] Trial 2 finished with value: 0.7984065138540224 and parameters: {'lgbm': 0.7639801017012084, 'cat': 0.8327638679180213, 'xgb': 0.3047555687750467}. Best is trial 0 with value: 0.798491824785957.
[I 2022-08-12 04:00:10,210] Trial 3 finished with value: 0.7976893823623447 and parameters: {'lgbm': 0.8049666995051481, 'cat': 0.026732946370137234, 'xgb': 0.30316623449819213}. Best is trial 0 with value: 0.79849182

[I 2022-08-12 04:00:16,939] Trial 35 finished with value: 0.7985437597733309 and parameters: {'lgbm': 0.7175348236673169, 'cat': 0.716370776428261, 'xgb': 0.7405823944409025}. Best is trial 35 with value: 0.7985437597733309.
[I 2022-08-12 04:00:17,151] Trial 36 finished with value: 0.7983327948598351 and parameters: {'lgbm': 0.7227506181437522, 'cat': 0.7327656558762814, 'xgb': 0.5647383774719754}. Best is trial 35 with value: 0.7985437597733309.
[I 2022-08-12 04:00:17,363] Trial 37 finished with value: 0.7983938956815011 and parameters: {'lgbm': 0.8926437922734589, 'cat': 0.623830434325471, 'xgb': 0.7141784287036523}. Best is trial 35 with value: 0.7985437597733309.
[I 2022-08-12 04:00:17,574] Trial 38 finished with value: 0.7982983438502596 and parameters: {'lgbm': 0.6243878964518375, 'cat': 0.5369169859519456, 'xgb': 0.4797929963364884}. Best is trial 35 with value: 0.7985437597733309.
[I 2022-08-12 04:00:17,784] Trial 39 finished with value: 0.7983622506373542 and parameters: {'lgb

[I 2022-08-12 04:00:24,469] Trial 70 finished with value: 0.7984220090881741 and parameters: {'lgbm': 0.5877154713752614, 'cat': 0.7865915166601728, 'xgb': 0.9133075262759145}. Best is trial 54 with value: 0.7986101161043626.
[I 2022-08-12 04:00:24,680] Trial 71 finished with value: 0.7985438680840589 and parameters: {'lgbm': 0.7175125342820229, 'cat': 0.7195550323006918, 'xgb': 0.7467439018124344}. Best is trial 54 with value: 0.7986101161043626.
[I 2022-08-12 04:00:24,892] Trial 72 finished with value: 0.7983703826474102 and parameters: {'lgbm': 0.5558906542578375, 'cat': 0.6715455033102633, 'xgb': 0.6908681185142225}. Best is trial 54 with value: 0.7986101161043626.
[I 2022-08-12 04:00:25,105] Trial 73 finished with value: 0.7984811075850573 and parameters: {'lgbm': 0.715959126565723, 'cat': 0.7538553505037522, 'xgb': 0.7955477589364335}. Best is trial 54 with value: 0.7986101161043626.
[I 2022-08-12 04:00:25,317] Trial 74 finished with value: 0.7986102419595973 and parameters: {'lg

[I 2022-08-12 04:00:31,929] Trial 105 finished with value: 0.7984609893759044 and parameters: {'lgbm': 0.7251539802206999, 'cat': 0.5825677093243656, 'xgb': 0.9306894861985788}. Best is trial 102 with value: 0.7986241134557959.
[I 2022-08-12 04:00:32,142] Trial 106 finished with value: 0.7983803092270394 and parameters: {'lgbm': 0.8233739780547458, 'cat': 0.5543189078608602, 'xgb': 0.7537266614274285}. Best is trial 102 with value: 0.7986241134557959.
[I 2022-08-12 04:00:32,366] Trial 107 finished with value: 0.7985767957883392 and parameters: {'lgbm': 0.6933005679905458, 'cat': 0.6415694305125541, 'xgb': 0.8034206188663908}. Best is trial 102 with value: 0.7986241134557959.
[I 2022-08-12 04:00:32,585] Trial 108 finished with value: 0.7985524948351517 and parameters: {'lgbm': 0.6955623524502257, 'cat': 0.6377496776367209, 'xgb': 0.856186544339637}. Best is trial 102 with value: 0.7986241134557959.
[I 2022-08-12 04:00:32,797] Trial 109 finished with value: 0.7985193666921034 and paramet

[I 2022-08-12 04:00:39,413] Trial 140 finished with value: 0.79843491234536 and parameters: {'lgbm': 0.9629728890830773, 'cat': 0.7761741190118349, 'xgb': 0.8893117085498655}. Best is trial 113 with value: 0.7986355033074978.
[I 2022-08-12 04:00:39,661] Trial 141 finished with value: 0.7985687324650212 and parameters: {'lgbm': 0.7887451431476943, 'cat': 0.7058526451421067, 'xgb': 0.9146106224808122}. Best is trial 113 with value: 0.7986355033074978.
[I 2022-08-12 04:00:39,870] Trial 142 finished with value: 0.7985043318144656 and parameters: {'lgbm': 0.815739178957732, 'cat': 0.734011772350047, 'xgb': 0.8767691216545551}. Best is trial 113 with value: 0.7986355033074978.
[I 2022-08-12 04:00:40,082] Trial 143 finished with value: 0.7984584072649792 and parameters: {'lgbm': 0.8429804743221843, 'cat': 0.7160446130501793, 'xgb': 0.8940758933879916}. Best is trial 113 with value: 0.7986355033074978.
[I 2022-08-12 04:00:40,291] Trial 144 finished with value: 0.7984982051918823 and parameters

...

[I 2022-08-12 04:04:36,148] Trial 1218 finished with value: 0.7985986882189418 and parameters: {'lgbm': 0.7405283048150062, 'cat': 0.7356802104765028, 'xgb': 0.9687577861973121}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:36,371] Trial 1219 finished with value: 0.798543840046671 and parameters: {'lgbm': 0.774473130866938, 'cat': 0.7128408942768848, 'xgb': 0.9377209994953435}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:36,593] Trial 1220 finished with value: 0.7985932203946446 and parameters: {'lgbm': 0.8632537473863823, 'cat': 0.803215605023622, 'xgb': 0.978964850261383}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:36,816] Trial 1221 finished with value: 0.798559439813705 and parameters: {'lgbm': 0.7184463274367101, 'cat': 0.7597370352313672, 'xgb': 0.9119945702676456}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:37,036] Trial 1222 finished with value: 0.7985312717691322 and pa

[I 2022-08-12 04:04:43,820] Trial 1252 finished with value: 0.7985680294969708 and parameters: {'lgbm': 0.7617708824084363, 'cat': 0.7800947567601809, 'xgb': 0.9429614911201358}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:44,085] Trial 1253 finished with value: 0.7985297939220037 and parameters: {'lgbm': 0.8017238966176912, 'cat': 0.8127138789842591, 'xgb': 0.9629531741877468}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:44,343] Trial 1254 finished with value: 0.7985517104793963 and parameters: {'lgbm': 0.8554595297995558, 'cat': 0.7697272914799534, 'xgb': 0.980535358949467}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:44,567] Trial 1255 finished with value: 0.7985507709450772 and parameters: {'lgbm': 0.7777216135124633, 'cat': 0.8402416936913618, 'xgb': 0.9983693039462326}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:44,789] Trial 1256 finished with value: 0.7985637546001509 an

[I 2022-08-12 04:04:51,683] Trial 1286 finished with value: 0.7984285667752511 and parameters: {'lgbm': 0.8714297741732178, 'cat': 0.7457033974280513, 'xgb': 0.9096541967043366}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:51,903] Trial 1287 finished with value: 0.7984053513057233 and parameters: {'lgbm': 0.5847742266716456, 'cat': 0.7914040905311276, 'xgb': 0.9427346609099666}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:52,124] Trial 1288 finished with value: 0.7985943965199782 and parameters: {'lgbm': 0.7632320443755031, 'cat': 0.7638509805514454, 'xgb': 0.9994741037786671}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:52,345] Trial 1289 finished with value: 0.7985551536824618 and parameters: {'lgbm': 0.740798738169027, 'cat': 0.8126116018697317, 'xgb': 0.9810733429107055}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:52,567] Trial 1290 finished with value: 0.7984018413403202 an

[I 2022-08-12 04:04:59,389] Trial 1320 finished with value: 0.7986314924938596 and parameters: {'lgbm': 0.7697696588180414, 'cat': 0.7817818367764691, 'xgb': 0.9652495985768728}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:59,610] Trial 1321 finished with value: 0.7983789609725255 and parameters: {'lgbm': 0.7895885713144365, 'cat': 0.8785817112110846, 'xgb': 0.8887158252658212}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:04:59,834] Trial 1322 finished with value: 0.7985684410628893 and parameters: {'lgbm': 0.7167057793220276, 'cat': 0.7440544668822436, 'xgb': 0.926044726289643}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:05:00,055] Trial 1323 finished with value: 0.7983023982414126 and parameters: {'lgbm': 0.7541231272384119, 'cat': 0.3060357297598545, 'xgb': 0.6127901623713363}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:05:00,277] Trial 1324 finished with value: 0.7985352239352906 an

[I 2022-08-12 04:05:07,063] Trial 1354 finished with value: 0.7986030242369628 and parameters: {'lgbm': 0.7355146323861175, 'cat': 0.7272536354623848, 'xgb': 0.9667642877252558}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:05:07,286] Trial 1355 finished with value: 0.7985804655110307 and parameters: {'lgbm': 0.7963607479834495, 'cat': 0.7526537422505477, 'xgb': 0.9077846714738986}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:05:07,507] Trial 1356 finished with value: 0.7986024292084588 and parameters: {'lgbm': 0.7636613491256069, 'cat': 0.7751761599412234, 'xgb': 0.9841540637953717}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:05:07,729] Trial 1357 finished with value: 0.798542068274198 and parameters: {'lgbm': 0.8136421496622295, 'cat': 0.8072997854806464, 'xgb': 0.9385289371334122}. Best is trial 1163 with value: 0.7986398820856142.
[I 2022-08-12 04:05:07,950] Trial 1358 finished with value: 0.7984663870972311 an

In [5]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgbm,params_xgb,state
1163,1163,0.798640,2022-08-12 04:04:23.562916,2022-08-12 04:04:23.784426,0 days 00:00:00.221510,0.777251,0.763293,0.958214,COMPLETE
326,326,0.798640,2022-08-12 04:01:19.369072,2022-08-12 04:01:19.581713,0 days 00:00:00.212641,0.769709,0.756518,0.947322,COMPLETE
281,281,0.798640,2022-08-12 04:01:09.645701,2022-08-12 04:01:09.857003,0 days 00:00:00.211302,0.751134,0.769920,0.918184,COMPLETE
1067,1067,0.798636,2022-08-12 04:04:01.824419,2022-08-12 04:04:02.044407,0 days 00:00:00.219988,0.771215,0.751158,0.982097,COMPLETE
279,279,0.798636,2022-08-12 04:01:09.218829,2022-08-12 04:01:09.430853,0 days 00:00:00.212024,0.737544,0.733696,0.923558,COMPLETE


In [6]:
study.best_params

{'lgbm': 0.763293437479992,
 'cat': 0.7772508849664402,
 'xgb': 0.958214117947317}

In [7]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgbm,params_xgb,state
465,465,0.798492,2022-08-09 00:19:32.745135,2022-08-09 00:19:32.965512,0 days 00:00:00.220377,0.597479,0.302462,0.915381,COMPLETE
292,292,0.798483,2022-08-09 00:18:55.175461,2022-08-09 00:18:55.387493,0 days 00:00:00.212032,0.635229,0.315323,0.945189,COMPLETE
1184,1184,0.798478,2022-08-09 00:22:11.650207,2022-08-09 00:22:11.874853,0 days 00:00:00.224646,0.637297,0.319941,0.952052,COMPLETE
543,543,0.798478,2022-08-09 00:19:49.965382,2022-08-09 00:19:50.179678,0 days 00:00:00.214296,0.618282,0.310276,0.922728,COMPLETE
623,623,0.798475,2022-08-09 00:20:07.340744,2022-08-09 00:20:07.556088,0 days 00:00:00.215344,0.570043,0.281972,0.868663,COMPLETE


In [8]:
study.best_params

{'lgbm': 0.3024622788138, 'cat': 0.5974790793281348, 'xgb': 0.9153811694061348}

In [10]:
prefixes = ['oof_lgbm_baseline_5fold_seed42', 'oof_v3_dpv25fold_seed42', 
            'oof_v1_dpv25fold_seed98', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]
prefixes = ['test_lgbm_baseline_5fold_seed42',
            'lgb_v3_dpv25fold_seed42', 'lgb_v1_dpv25fold_seed98', 'lgb_v2_dpv25fold_seed42']
submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    p_xgb = trial.suggest_uniform('xgb', 0.0, 1.0)
    p_lgb = trial.suggest_uniform('lgb', 0.0, 1.0)
    p_lgbb = trial.suggest_uniform('lgbb', 0.0, 1.0)
    y = valids[0]['target'].values
    a_cat = valids[0].drop(['target','customer_ID'], axis=1).values
    a_xgb = valids[1].drop(['target','customer_ID'], axis=1).values
    a_lgb = valids[2].drop(['target','customer_ID'], axis=1).values
    a_lgbb = valids[3].drop(['target','customer_ID'], axis=1).values
    a = (p_cat * a_cat + p_xgb * a_xgb + p_lgb * a_lgb + p_lgbb * a_lgbb)
    b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgb+p_lgbb)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=300)

[I 2022-08-12 04:20:49,956] A new study created in memory with name: no-name-f2a7a8c6-eb23-4f52-b3c0-efbea433de39
[I 2022-08-12 04:20:50,166] Trial 0 finished with value: 0.7979163728184107 and parameters: {'cat': 0.8650311318658788, 'xgb': 0.2256202384424374, 'lgb': 0.785859910200411, 'lgbb': 0.3599692247353602}. Best is trial 0 with value: 0.7979163728184107.
[I 2022-08-12 04:20:50,367] Trial 1 finished with value: 0.798178846896714 and parameters: {'cat': 0.7647322934802616, 'xgb': 0.6023314705166222, 'lgb': 0.4656221538893476, 'lgbb': 0.37314314141141947}. Best is trial 1 with value: 0.798178846896714.
[I 2022-08-12 04:20:50,568] Trial 2 finished with value: 0.798376463325122 and parameters: {'cat': 0.2646207370123067, 'xgb': 0.8370131891584057, 'lgb': 0.28423646082530063, 'lgbb': 0.4436519586685197}. Best is trial 2 with value: 0.798376463325122.
[I 2022-08-12 04:20:50,769] Trial 3 finished with value: 0.7982544975032355 and parameters: {'cat': 0.020629810653186254, 'xgb': 0.02094

[I 2022-08-12 04:20:56,719] Trial 31 finished with value: 0.7987029793269305 and parameters: {'cat': 0.5686923552284997, 'xgb': 0.9988917103116852, 'lgb': 0.7796488954838554, 'lgbb': 0.8174790585537421}. Best is trial 31 with value: 0.7987029793269305.
[I 2022-08-12 04:20:56,932] Trial 32 finished with value: 0.7985275287314876 and parameters: {'cat': 0.4863762914288612, 'xgb': 0.6415780229547002, 'lgb': 0.7238855015891127, 'lgbb': 0.8211007432864268}. Best is trial 31 with value: 0.7987029793269305.
[I 2022-08-12 04:20:57,149] Trial 33 finished with value: 0.7986396870248078 and parameters: {'cat': 0.564333251035301, 'xgb': 0.9254096608580477, 'lgb': 0.6331224365395781, 'lgbb': 0.7459074551957114}. Best is trial 31 with value: 0.7987029793269305.
[I 2022-08-12 04:20:57,363] Trial 34 finished with value: 0.7985344899710672 and parameters: {'cat': 0.5631778301912159, 'xgb': 0.9540722388909171, 'lgb': 0.7861779443224439, 'lgbb': 0.4879138159935741}. Best is trial 31 with value: 0.7987029

[I 2022-08-12 04:21:03,433] Trial 62 finished with value: 0.7986775329096327 and parameters: {'cat': 0.6264902343676872, 'xgb': 0.9708352581579743, 'lgb': 0.7840307991730355, 'lgbb': 0.7088670708795788}. Best is trial 31 with value: 0.7987029793269305.
[I 2022-08-12 04:21:03,660] Trial 63 finished with value: 0.7986684622495297 and parameters: {'cat': 0.6293643206018236, 'xgb': 0.9659314622353686, 'lgb': 0.7766544780464332, 'lgbb': 0.7015649992553259}. Best is trial 31 with value: 0.7987029793269305.
[I 2022-08-12 04:21:03,886] Trial 64 finished with value: 0.7986272315872299 and parameters: {'cat': 0.6644454484633986, 'xgb': 0.9660691011439516, 'lgb': 0.6754555837645371, 'lgbb': 0.7102860461857502}. Best is trial 31 with value: 0.7987029793269305.
[I 2022-08-12 04:21:04,111] Trial 65 finished with value: 0.7985381814936581 and parameters: {'cat': 0.7138641610990707, 'xgb': 0.9124558310737877, 'lgb': 0.7195589712829283, 'lgbb': 0.7760733212850188}. Best is trial 31 with value: 0.798702

[I 2022-08-12 04:21:10,294] Trial 93 finished with value: 0.7986055807668606 and parameters: {'cat': 0.5073184815515627, 'xgb': 0.9778385774845569, 'lgb': 0.6939302078643238, 'lgbb': 0.9683731662971602}. Best is trial 73 with value: 0.7987636605404145.
[I 2022-08-12 04:21:10,521] Trial 94 finished with value: 0.7986032515797512 and parameters: {'cat': 0.5372215162296081, 'xgb': 0.891761498576127, 'lgb': 0.6378124654994839, 'lgbb': 0.8439495232127333}. Best is trial 73 with value: 0.7987636605404145.
[I 2022-08-12 04:21:10,745] Trial 95 finished with value: 0.7986399752075733 and parameters: {'cat': 0.4272887131050919, 'xgb': 0.9269981158337635, 'lgb': 0.6720231613021698, 'lgbb': 0.8137876797539622}. Best is trial 73 with value: 0.7987636605404145.
[I 2022-08-12 04:21:10,977] Trial 96 finished with value: 0.7985005141734651 and parameters: {'cat': 0.4768100762802082, 'xgb': 0.9746666236238045, 'lgb': 0.7010782212706282, 'lgbb': 0.4092307810728149}. Best is trial 73 with value: 0.7987636

[I 2022-08-12 04:21:17,047] Trial 124 finished with value: 0.7986642717808203 and parameters: {'cat': 0.4707211470639132, 'xgb': 0.9391691763872685, 'lgb': 0.7120207605651243, 'lgbb': 0.8660241610555484}. Best is trial 73 with value: 0.7987636605404145.
[I 2022-08-12 04:21:17,263] Trial 125 finished with value: 0.7987082819004552 and parameters: {'cat': 0.43800339183017223, 'xgb': 0.9736269689375544, 'lgb': 0.6129309035135114, 'lgbb': 0.7747472381159788}. Best is trial 73 with value: 0.7987636605404145.
[I 2022-08-12 04:21:17,479] Trial 126 finished with value: 0.7985679587044102 and parameters: {'cat': 0.33582417270583886, 'xgb': 0.9692037276380226, 'lgb': 0.6610884730445915, 'lgbb': 0.7667586486302588}. Best is trial 73 with value: 0.7987636605404145.
[I 2022-08-12 04:21:17,695] Trial 127 finished with value: 0.7986534122483346 and parameters: {'cat': 0.40082654812330865, 'xgb': 0.8694194977708102, 'lgb': 0.6109671499655588, 'lgbb': 0.793694279395834}. Best is trial 73 with value: 0.

...

[I 2022-08-12 04:25:16,959] Trial 1178 finished with value: 0.79874606854227 and parameters: {'cat': 0.3417119078894625, 'xgb': 0.9066656031804698, 'lgb': 0.49397232187215434, 'lgbb': 0.9165768541325099}. Best is trial 1176 with value: 0.7988105115578148.
[I 2022-08-12 04:25:17,192] Trial 1179 finished with value: 0.7987649454953092 and parameters: {'cat': 0.37264781190977747, 'xgb': 0.9201961887236502, 'lgb': 0.4839921917418883, 'lgbb': 0.9336891680702168}. Best is trial 1176 with value: 0.7988105115578148.
[I 2022-08-12 04:25:17,425] Trial 1180 finished with value: 0.798700588395882 and parameters: {'cat': 0.3350238083646594, 'xgb': 0.8950123189301082, 'lgb': 0.5001893315350262, 'lgbb': 0.9198172272581576}. Best is trial 1176 with value: 0.7988105115578148.
[I 2022-08-12 04:25:17,656] Trial 1181 finished with value: 0.7987863036960978 and parameters: {'cat': 0.3091554418286059, 'xgb': 0.8767999183660269, 'lgb': 0.4785595802452569, 'lgbb': 0.9333956406259399}. Best is trial 1176 with 

[I 2022-08-12 04:25:24,252] Trial 1209 finished with value: 0.7986931434846749 and parameters: {'cat': 0.2304412585715391, 'xgb': 0.8373336772728017, 'lgb': 0.4232421801494012, 'lgbb': 0.9442411266772925}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:24,482] Trial 1210 finished with value: 0.7987087575042533 and parameters: {'cat': 0.19857879510787046, 'xgb': 0.8069126907640513, 'lgb': 0.4577488802022142, 'lgbb': 0.8818961776455286}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:24,713] Trial 1211 finished with value: 0.7986626490475843 and parameters: {'cat': 0.2833147540477125, 'xgb': 0.8211095287605986, 'lgb': 0.4793729852292666, 'lgbb': 0.9545527171872438}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:24,944] Trial 1212 finished with value: 0.7986984755811335 and parameters: {'cat': 0.26398046836324723, 'xgb': 0.8355068973426846, 'lgb': 0.4045738162031425, 'lgbb': 0.9278016090048539}. Best is trial 1190 wi

[I 2022-08-12 04:25:31,513] Trial 1240 finished with value: 0.798721322145856 and parameters: {'cat': 0.2407061601351035, 'xgb': 0.8089674430979086, 'lgb': 0.40804500296057283, 'lgbb': 0.9062029504614519}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:31,745] Trial 1241 finished with value: 0.7986253611968521 and parameters: {'cat': 0.2015466203796169, 'xgb': 0.7746151403255753, 'lgb': 0.4545426415802319, 'lgbb': 0.9355548663270533}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:31,976] Trial 1242 finished with value: 0.7987694025423866 and parameters: {'cat': 0.29953785305076175, 'xgb': 0.8243414523787462, 'lgb': 0.4509263760666562, 'lgbb': 0.9081959312655404}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:32,205] Trial 1243 finished with value: 0.7987241606441358 and parameters: {'cat': 0.21438785229903456, 'xgb': 0.8101377549644138, 'lgb': 0.4309757369250937, 'lgbb': 0.9234642539437813}. Best is trial 1190 wi

[I 2022-08-12 04:25:38,873] Trial 1271 finished with value: 0.7987701207527986 and parameters: {'cat': 0.23607790712766474, 'xgb': 0.8501494361709439, 'lgb': 0.47936029398754554, 'lgbb': 0.9229315269154299}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:39,108] Trial 1272 finished with value: 0.7987244758761124 and parameters: {'cat': 0.22065977738098716, 'xgb': 0.8286129927501787, 'lgb': 0.45338555519890683, 'lgbb': 0.9371723196085315}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:39,341] Trial 1273 finished with value: 0.7986872760712451 and parameters: {'cat': 0.2777135834959651, 'xgb': 0.8101983099809363, 'lgb': 0.4468002288113154, 'lgbb': 0.9441287791038826}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:39,573] Trial 1274 finished with value: 0.7987228293402384 and parameters: {'cat': 0.25602621197596837, 'xgb': 0.8372332185831531, 'lgb': 0.4852603576386986, 'lgbb': 0.9144606420474723}. Best is trial 1190

[I 2022-08-12 04:25:46,201] Trial 1302 finished with value: 0.7986366777517827 and parameters: {'cat': 0.14783207454496394, 'xgb': 0.8253279737783291, 'lgb': 0.46670162319510444, 'lgbb': 0.9740958554851862}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:46,434] Trial 1303 finished with value: 0.7986422239225588 and parameters: {'cat': 0.31540475982572114, 'xgb': 0.8513602819439761, 'lgb': 0.4314365520475369, 'lgbb': 0.9988815199287135}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:46,669] Trial 1304 finished with value: 0.7987510445256676 and parameters: {'cat': 0.2766651510072164, 'xgb': 0.8359347195657595, 'lgb': 0.47121080947621863, 'lgbb': 0.951449215715726}. Best is trial 1190 with value: 0.7988276968863532.
[I 2022-08-12 04:25:46,906] Trial 1305 finished with value: 0.798715328901937 and parameters: {'cat': 0.2943758411381716, 'xgb': 0.8592730447196234, 'lgb': 0.45074944233572284, 'lgbb': 0.958268535619619}. Best is trial 1190 wi

In [11]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgb,params_lgbb,params_xgb,state
1190,1190,0.798828,2022-08-12 04:25:19.560887,2022-08-12 04:25:19.791625,0 days 00:00:00.230738,0.249174,0.455044,0.928754,0.836889,COMPLETE
1176,1176,0.798811,2022-08-12 04:25:16.266489,2022-08-12 04:25:16.495811,0 days 00:00:00.229322,0.394859,0.521583,0.914006,0.913367,COMPLETE
1253,1253,0.798803,2022-08-12 04:25:34.368028,2022-08-12 04:25:34.604704,0 days 00:00:00.236676,0.243093,0.465788,0.948569,0.842105,COMPLETE
1288,1288,0.798803,2022-08-12 04:25:42.686055,2022-08-12 04:25:42.917703,0 days 00:00:00.231648,0.249926,0.452716,0.948989,0.833615,COMPLETE
1293,1293,0.798795,2022-08-12 04:25:43.856158,2022-08-12 04:25:44.088881,0 days 00:00:00.232723,0.237640,0.448816,0.936694,0.830670,COMPLETE


In [12]:
study.best_params

{'cat': 0.24917444640087694,
 'xgb': 0.836888904619204,
 'lgb': 0.45504415528615116,
 'lgbb': 0.928754329257015}

In [14]:
# {'cat': 0.24917444640087694,
#  'xgb': 0.836888904619204,
#  'lgb': 0.45504415528615116,
#  'lgbb': 0.928754329257015}

p_cat = study.best_params['cat']
p_xgb = study.best_params['xgb']
p_lgb = study.best_params['lgb']
p_lgbb = study.best_params['lgbb']

a_cat = submissions[0].drop(['customer_ID'], axis=1).values
a_xgb = submissions[1].drop(['customer_ID'], axis=1).values
a_lgb = submissions[2].drop(['customer_ID'], axis=1).values
a_lgbb = submissions[3].drop(['customer_ID'], axis=1).values

a = (p_cat * a_cat + p_xgb * a_xgb + p_lgb * a_lgb + p_lgbb * a_lgbb)
b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgb+p_lgbb)

sub = submissions[0][['customer_ID']].reset_index(drop=True)
sub['prediction'] = b

In [15]:
sub.to_csv('blend_99_98_98_97_cv798828.csv', index=False)
!kaggle competitions submit -c amex-default-prediction -f blend_99_98_98_97_cv798828.csv -m "blend 4 models CV 798828"

100%|██████████████████████████████████████| 75.3M/75.3M [00:07<00:00, 10.8MB/s]
Successfully submitted to American Express - Default Prediction

In [10]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_xgb,state
912,912,0.798149,2022-08-09 00:27:44.697441,2022-08-09 00:27:44.907271,0 days 00:00:00.209830,0.376803,0.924976,COMPLETE
447,447,0.798149,2022-08-09 00:26:05.621914,2022-08-09 00:26:05.831747,0 days 00:00:00.209833,0.386515,0.949753,COMPLETE
357,357,0.798149,2022-08-09 00:25:46.422990,2022-08-09 00:25:46.633638,0 days 00:00:00.210648,0.390533,0.960053,COMPLETE
666,666,0.798149,2022-08-09 00:26:52.006650,2022-08-09 00:26:52.214060,0 days 00:00:00.207410,0.346048,0.850771,COMPLETE
127,127,0.798149,2022-08-09 00:24:57.425699,2022-08-09 00:24:57.628382,0 days 00:00:00.202683,0.357208,0.878219,COMPLETE


In [11]:
study.best_params

{'cat': 0.37680278489027397, 'xgb': 0.9249755054215106}

In [16]:
best_params = {'lgbm': 0.3024622788138, 'cat': 0.5974790793281348, 'xgb': 0.9153811694061348}
best_params

{'lgbm': 0.3024622788138, 'cat': 0.5974790793281348, 'xgb': 0.9153811694061348}

In [17]:
p_lgbm = best_params['lgbm']
p_cat = best_params['cat']
p_xgb = best_params['xgb']

a_lgbm = submissions[0].drop(['customer_ID'], axis=1).values
a_cat = submissions[1].drop(['customer_ID'], axis=1).values
a_xgb = submissions[2].drop(['customer_ID'], axis=1).values
a = p_lgbm * a_lgbm + p_cat * a_cat + p_xgb * a_xgb
b = a.reshape(len(a_lgbm))/(p_lgbm+p_cat+p_xgb)

sub = submissions[0][['customer_ID']].reset_index(drop=True)
sub['prediction'] = b

In [18]:
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.027030
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000957
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.030819
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.182994
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.862092
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.014632
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.848909
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.445129
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.263341


In [20]:
sub.to_csv('blend_99_98_97_cv985.csv', index=False)

In [21]:
!kaggle competitions submit -c amex-default-prediction -f blend_99_98_97_cv985.csv -m "blend 3 models CV 7985"

100%|██████████████████████████████████████| 75.3M/75.3M [00:05<00:00, 15.5MB/s]
Successfully submitted to American Express - Default Prediction

In [26]:
a = valids[0].drop(['target','customer_ID'], axis=1)
a = a.rename(columns={col: f'{col}_0' for col in a.columns})
b = valids[1].drop(['target','customer_ID'], axis=1)
b = b.rename(columns={col: f'{col}_1' for col in b.columns})
c = valids[2].drop(['target','customer_ID'], axis=1)
c = c.rename(columns={col: f'{col}_2' for col in c.columns})

mgd_val = pd.concat([a, b, c], axis=1)
mgd_val['target'] = valids[0]['target']
mgd_val['customer_ID'] = valids[0]['customer_ID']
                   
a = submissions[0].drop(['customer_ID'], axis=1)
a = a.rename(columns={col: f'{col}_0' for col in a.columns})
b = submissions[1].drop(['customer_ID'], axis=1)
b = b.rename(columns={col: f'{col}_1' for col in b.columns})
c = submissions[2].drop(['customer_ID'], axis=1)
c = c.rename(columns={col: f'{col}_2' for col in c.columns})

mgd_sub = pd.concat([a, b, c], axis=1)
mgd_sub['customer_ID'] = submissions[0]['customer_ID']

In [27]:
mgd_val.shape, mgd_sub.shape

((458913, 5), (924621, 4))

In [28]:
mgd_val.to_csv('train_merged.csv', index=False)
mgd_sub.to_csv('test_merged.csv', index=False)